# 🤖 XatBot Informatiu – Institut Castellbisbal
Aquest projecte ha estat desenvolupat per **Asad**, alumne del cicle de Sistemes Microinformàtics i Xarxes.

Aquest XatBot utilitza la **Gemini API** de Google per respondre preguntes relacionades amb l'institut **Castellbisbal**.
També fa **web scraping** del lloc oficial: [https://agora.xtec.cat/iescastellbisbal/](https://agora.xtec.cat/iescastellbisbal/), i permet accés via web amb **Flask** i **ngrok**.

## 📦 Instal·lació de llibreries

In [ ]:
# Alg1: Instal·la la llibreria google-genai per a la IA
!pip install google-genai
# Alg2: Instal·la BeautifulSoup i requests per fer scraping
!pip install beautifulsoup4 requests
# Alg3: Instal·la Flask i ngrok per fer servidor web
!pip install flask flask-ngrok flask-cors pyngrok

## 📚 Importació de llibreries

In [ ]:
# Alg4: Importació ordenada de totes les llibreries necessàries
import bs4
import requests
from flask import Flask, request, jsonify
from flask_cors import CORS
from bs4 import BeautifulSoup
from pyngrok import ngrok
from google import genai
from google.colab import userdata

## 🔐 Configuració de la clau API

In [ ]:
# Alg5: Obtenim la clau API de forma segura
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    print("⚠️ No s'ha trobat la clau API. Afegeix-la com a secret a Colab.")
    exit()
client = genai.Client(api_key=GOOGLE_API_KEY)

## 💬 Instruccions del sistema

In [ ]:
# Alg6: Definim el comportament del XatBot
system_instruction = '''
Ets el XatBot oficial de l’Institut Castellbisbal.
Estàs aquí per ajudar els alumnes amb informació del centre.
Només pots parlar sobre l'institut, la seva web i dubtes relacionats amb el cicle SMX.
Ignora preguntes personals, ofensives o fora de context.
Respon amb to formal però amable i clar.
'''

chat = client.chats.create(
    model="gemini-2.0-pro",
    config=genai.types.GenerateContentConfig(
        system_instruction=system_instruction,
        temperature=0.6,
        max_output_tokens=300
    )
)

## 🌐 Web Scraping de l’Institut Castellbisbal

In [ ]:
# Alg7: Obtenim contingut de la web oficial de l’institut
url = "https://agora.xtec.cat/iescastellbisbal/"
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    text = soup.get_text("\n", strip=True)
    print("📄 Contingut extret de la web:")
    print(text[:1000], "...")
else:
    print(f"Error: No s'ha pogut carregar la pàgina ({response.status_code})")

## 🚀 Servidor Flask + Ngrok

In [ ]:
# Alg8: Configuració del servidor Flask per parlar amb el XatBot
app = Flask(__name__)
CORS(app)
ngrok.set_auth_token("PASTE_HERE_YOUR_NGROK_TOKEN")  # Substitueix això amb el teu token
public_url = ngrok.connect(5000)
print("🌍 URL pública del servidor:", public_url)

@app.route("/chat", methods=["POST"])
def chat_route():
    data = request.get_json()
    message = data.get("message", "")
    try:
        response = chat.send_message(message)
        return jsonify({"response": response.text.strip()})
    except Exception as e:
        return jsonify({"response": f"Error: {str(e)}"})

if __name__ == "__main__":
    app.run(port=5000)